# 1. 데이터 불러오기

In [1]:
from google.colab import files
myfile = files.upload()


Saving conjoint.xlsx to conjoint.xlsx


In [2]:
import io
import pandas as pd
df = pd.read_excel('conjoint.xlsx')

In [3]:
import numpy as np

In [4]:
df

,작동시간,AS기간,가격,김경호,김성윤,김호준,이호빈,양원준
0,10,12,8,3,5,8,5,3
1,10,12,14,10,6,5,7,4
2,10,12,30,1,1,2,10,7
3,10,24,8,4,2,9,4,9
4,10,24,14,4,9,7,10,10
5,10,24,30,8,5,2,2,10
6,10,36,8,8,6,4,5,5
7,10,36,14,1,1,10,7,7
8,10,36,30,1,6,5,7,7
9,15,12,8,1,4,4,1,7


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   작동시간    27 non-null     int64
 1   AS기간    27 non-null     int64
 2   가격      27 non-null     int64
 3   김경호     27 non-null     int64
 4   김성윤     27 non-null     int64
 5   김호준     27 non-null     int64
 6   이호빈     27 non-null     int64
 7   양원준     27 non-null     int64
dtypes: int64(8)
memory usage: 1.8 KB


# 2.속성-레벨 별 효용가치 출력 (0:김경호 1:김성윤 2:김호준 3:이호빈 4:양원준)

In [6]:
import statsmodels.api as sm
from sympy import *


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
con_df = pd.get_dummies(df, columns=['작동시간', 'AS기간', '가격'], drop_first = False)
con_df['intercept'] = 1.0
# y = [작동시간_15, AS기간_24, 가격_14]
X = con_df[['intercept', '작동시간_10', '작동시간_15','AS기간_12', 'AS기간_36', '가격_8', '가격_30']]
res = []
intercept = []

for i in range(5):
    Y = con_df[con_df.columns[0:5][i]]
    LR1 = sm.OLS(Y,X).fit()
    x, y, z = symbols('a_1 a_2 a_3')
    a1=solve( [Eq(x-z,LR1.params[1]), Eq(y-z,LR1.params[2]), Eq(x+y+z, 0)], [x,y,z] )
    x, y = symbols('b_1 b_2')
    b1=solve( [Eq(x-z,LR1.params[3]), Eq(y-z,LR1.params[4]), Eq(x+y+z, 0)], [x,y,z] )
    x, y = symbols('c_1 c_2')
    c1=solve( [Eq(x-z,LR1.params[5]), Eq(y-z,LR1.params[6]), Eq(x+y+z, 0)], [x,y,z] )
    a_1=list(a1.values())
    b_1=list(b1.values())
    c_1=list(c1.values())
    a_1.extend(b_1)
    a_1.extend(c_1)
    res.append(a_1)
    d_1 = LR1.params[0]
    intercept.append(d_1)

result = pd.DataFrame(res)
result.columns = ['작동시간_10', '작동시간_15', '작동시간_20', 'AS기간_12', 'AS기간_24', 'AS기간_36', '가격_8', '가격_14', '가격_30']
result.head()

    

,작동시간_10,작동시간_15,작동시간_20,AS기간_12,AS기간_24,AS기간_36,가격_8,가격_14,가격_30
0,-0.962962962962963,0.592592592592593,0.370370370370370,0.703703703703704,0.0370370370370370,-0.740740740740741,-0.185185185185185,-0.407407407407407,0.592592592592593
1,-1.03703703703704,0.296296296296296,0.740740740740741,-0.370370370370370,-0.370370370370370,0.740740740740741,-0.481481481481481,-0.0370370370370370,0.518518518518518
2,1.00000000000000,-0.888888888888889,-0.111111111111111,-0.222222222222221,0.111111111111110,0.111111111111112,-0.666666666666667,0.666666666666667,0.0
3,1.00000000000000,-1.11111111111111,0.111111111111111,-1.44444444444444,1.44444444444444,0.0,-1.44444444444444,0.0,1.44444444444444
4,0.814814814814815,-0.0740740740740741,-0.740740740740741,-0.185185185185185,0.259259259259259,-0.0740740740740741,-0.407407407407407,0.592592592592593,-0.185185185185185


# 3. 각 경우의 수 별 효용 (0:김경호 1:김성윤 2:김호준 3:이호빈 4:양원준)

In [8]:
import numpy as np
con_df2 = con_df[['작동시간_10', '작동시간_15', '작동시간_20', 'AS기간_12', 'AS기간_24', 'AS기간_36', '가격_8', '가격_14', '가격_30']]
alt1 = []
for i in range(0,27,1):
    alt1.append(con_df2.loc[i])
alt2 = []
for i in range(0,27,1):
    for k in range(0,5,1):
        alt2.append(np.dot(alt1[i], result.loc[k]))

utility = []
for i in range(0,134,5):
    utility.append(alt2[i:i+5])
utility = pd.DataFrame(utility).T
utility.columns = ['S1', 'S2', 'S3', 'S4' , 'S5' , 'S6' , 'S7' , 'S8' , 'S9' , 'S10' , 'S11' , 'S12' , 'S13' , 'S14' , 'S15' , 'S16' , 'S17' , 'S18' , 'S19' , 'S20' , 'S21' , 'S22' , 'S23' , 'S24' , 'S25' , 'S26', 's27' ]
utility = utility + np.array(intercept).mean()
print(utility.head())


                 S1                S2  ...               S26               s27
0  5.54814814814815  5.32592592592593  ...  5.21481481481482  6.21481481481482
1  4.10370370370370  4.54814814814815  ...  7.43703703703704  7.99259259259259
2  6.10370370370371  7.43703703703704  ...  6.65925925925926  5.99259259259259
3  4.10370370370370  5.54814814814815  ...  6.10370370370370  7.54814814814815
4  6.21481481481482  7.21481481481482  ...  5.77037037037037  4.99259259259259

[5 rows x 27 columns]


# 4. 순위, 중요도, 효용 (0:김경호 1:김성윤 2:김호준 3:이호빈 4:양원준)

In [9]:
import pandas as pd

alt_mean = pd.DataFrame(utility.mean(), columns=['효용'])
alt_mean['순위'] = alt_mean['효용'].rank(ascending=False)
print(alt_mean.sort_values(by=['순위'],ascending=True))

a_dif=max(result.mean()[0:3]-min(result.mean()[0:3]))
b_dif=max(result.mean()[3:6]-min(result.mean()[3:6]))
c_dif=max(result.mean()[6:9]-min(result.mean()[6:9]))

print("\n작동시간 중요도: ", round(a_dif/(a_dif+b_dif+c_dif)*100,2))
print("AS기간 중요도: ", round(b_dif/(a_dif+b_dif+c_dif)*100,2))
print("가격 중요도: ", round(c_dif/(a_dif+b_dif+c_dif)*100,2))

print("\n작동시간 효용(10/15/20):\n",result.mean()[0:3])
print("\nAS기간 효용(12/24/36):\n",result.mean()[3:6])
print("\n가격 효용(8/14/30):\n",result.mean()[6:9])



           효용    순위
S6   6.925926   1.0
S24  6.837037   2.0
S9   6.637037   3.0
S5   6.614815   4.0
s27  6.548148   5.0
S23  6.525926   6.5
S15  6.525926   6.5
S8   6.325926   8.0
S3   6.325926   9.0
S26  6.237037  10.0
S21  6.237037  11.5
S18  6.237037  11.5
S14  6.214815  13.0
S2   6.014815  14.0
S17  5.925926  15.5
S20  5.925926  15.5
S12  5.925926  17.0
S4   5.814815  18.0
S22  5.725926  19.0
S11  5.614815  20.0
S7   5.525926  21.0
S25  5.437037  22.0
S13  5.414815  23.0
S1   5.214815  24.0
S19  5.125926  25.0
S16  5.125926  26.0
S10  4.814815  27.0

작동시간 중요도:  18.95
AS기간 중요도:  28.42
가격 중요도:  52.63

작동시간 효용(10/15/20):
 작동시간_10    0.162963
작동시간_15   -0.237037
작동시간_20    0.074074
dtype: float64

AS기간 효용(12/24/36):
 AS기간_12   -0.303704
AS기간_24    0.296296
AS기간_36    0.007407
dtype: float64

가격 효용(8/14/30):
 가격_8    -0.637037
가격_14    0.162963
가격_30    0.474074
dtype: float64


# 5. 사람별 선택한 경우의 수 (0:김경호 1:김성윤 2:김호준 3:이호빈 4:양원준)

(한명당 가장 좋은 선택지 한 개 선택)

In [10]:
pre1 = []
for i in range(0,5,1):
    sum1 = np.where(utility.loc[i].values==utility.loc[i].max(),1,0)
    pre1.append(sum1)

max = pd.DataFrame(pre1)
max.columns = utility.columns
print(max)


   S1  S2  S3  S4  S5  S6  S7  S8  ...  S20  S21  S22  S23  S24  S25  S26  s27
0   0   0   0   0   0   0   0   0  ...    0    0    0    0    0    0    0    0
1   0   0   0   0   0   0   0   0  ...    0    0    0    0    0    0    0    1
2   0   0   0   0   0   0   0   1  ...    0    0    0    0    0    0    0    0
3   0   0   0   0   0   1   0   0  ...    0    0    0    0    0    0    0    0
4   0   0   0   0   1   0   0   0  ...    0    0    0    0    0    0    0    0

[5 rows x 27 columns]


# 5-1. 사람별 시장 점유율 예측 (0:김경호 1:김성윤 2:김호준 3:이호빈 4:양원준)
(한명당 가장 좋은 선택지 한 개 선택)


In [11]:
pre2 = []
for i in range(0,27,1):
    maxsum = max.iloc[:,i].sum()
    pre2.append(maxsum)

prefer = pd.DataFrame(pre2).T
prefer.columns = utility.columns
print(prefer/27*100)

    S1   S2   S3   S4        S5        S6  ...  S22  S23  S24  S25  S26       s27
0  0.0  0.0  0.0  0.0  3.703704  3.703704  ...  0.0  0.0  0.0  0.0  0.0  3.703704

[1 rows x 27 columns]


# 6. 사람별 선택 확률 (0:김경호 1:김성윤 2:김호준 3:이호빈 4:양원준)
(선택확률 기준)

In [12]:
pro1 = []
for i in range(0,5,1):
    sum2 = utility.loc[i]/utility.loc[i].sum()
    pro1.append(sum2)

select1 = round(pd.DataFrame(pro1).astype(float),3)
select1.columns = utility.columns
print(select1)

      S1     S2     S3     S4     S5  ...    S23    S24    S25    S26    s27
0  0.034  0.033  0.039  0.030  0.029  ...  0.037  0.043  0.034  0.032  0.038
1  0.025  0.028  0.032  0.025  0.028  ...  0.039  0.043  0.043  0.046  0.049
2  0.038  0.046  0.042  0.040  0.048  ...  0.041  0.037  0.033  0.041  0.037
3  0.025  0.034  0.043  0.043  0.052  ...  0.047  0.056  0.029  0.038  0.047
4  0.038  0.045  0.040  0.041  0.047  ...  0.038  0.033  0.029  0.036  0.031

[5 rows x 27 columns]


# 6-1. 사람별 시장 점유율 예측 (0:김경호 1:김성윤 2:김호준 3:이호빈 4:양원준)
(선택확률 기준)

In [13]:
pro2 = []
for i in range(0,27,1):
    sum3 = select1.iloc[:,i].sum()
    pro2.append(sum3)

select2 = round(pd.DataFrame(pro2).astype(float).T,3)
select2.columns = utility.columns
print(select2/27*100)

         S1        S2        S3  ...       S25       S26       s27
0  0.592593  0.688889  0.725926  ...  0.622222  0.714815  0.748148

[1 rows x 27 columns]
